In [1]:
#pip install pyLDAvis

In [2]:
import re
import sys
import time
import math
import pickle
import warnings
import numpy as np
import pandas as pd
import pyLDAvis.gensim
from textblob import TextBlob
import matplotlib.pyplot as plt

import nltk
import nltk.corpus  
from nltk.text import Text
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import sklearn
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer

/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, an

# Content #
1. Import and inspect data 
2. Preprocessing: tfidf tdm, tokenization, lemmatization, stemming
3. Machine Learning Classification
    - Logistic Regression
    - SVM
    - Discriminant
    - Naive Bayes
    - Random Forest
    - Boosting
    - Neural Network

**IMPORT and INSPECT DATA**

In [4]:
#articles = pd.read_csv("New Labled Data2.csv")
articles = pd.read_csv("/Users/jihunlee/Desktop/fulldata_complete_GPR_v2.csv")
articles.head(5)

,Type,Level,Entity,Title,Text,Publish_Date,URL,Scrape_Date,Source,Content,...,article_neg,article_neu,article_pos,tokenized_sents,extracted_gpes,loc1,loc2,loc3,GRE_list,Content_GRE_com
0,0,0.0,Germany,Man confesses to cyberattack on German politic...,A 20-year-old man arrested in connection with ...,1/8/19 0:00,https://www.cnn.com/2019/01/08/europe/arrest-g...,NaN,CNN,Man confesses to cyberattack on German politic...,...,0.0,1.000,0.000,"[""A 20-year-old man arrested in connection wit...","['Germany', 'Wiesbaden', 'Hesse', 'Frankfurt',...",GERMANY,WIESBADEN,HESSE,"['Germany', 'Wiesbaden', 'Hesse', 'Frankfurt']",1
1,0,0.0,Israeli,Anonymous Arab' cyberattacks hit Israel,Several Israeli government websites appeared t...,4/7/13 0:00,https://www.cnn.com/2013/04/07/world/meast/isr...,NaN,CNN,Anonymous Arab' cyberattacks hit Israel,...,0.0,1.000,0.000,['Several Israeli government websites appeared...,"['Israel', 'Israel', 'Yisrael', 'Israel']",ISRAEL,YISRAEL,NaN,"['Israel', 'Yisrael']",1
2,0,0.0,USA,Pentagon says China using cyberattacks,The Pentagon has accused China of trying to ex...,5/7/13 0:00,https://money.cnn.com/2013/05/07/news/china-cy...,NaN,CNN,Pentagon says China using cyberattacks,...,0.0,1.000,0.000,['The Pentagon has accused China of trying to ...,"['China', 'U.S.', 'U.S.', 'U.S.', 'China', 'U....",CHINA,U.S.,IRAN,"['China', 'U.S.', 'Iran', 'Virginia', 'Shanghai']",2
3,0,0.0,World,New startups prime targets for cyberattacks,Startups take note: Cybercriminals are onto yo...,5/23/13 0:00,https://money.cnn.com/2013/05/23/technology/st...,NaN,CNN,New startups prime targets for cyberattacks,...,0.0,1.000,0.000,['Startups take note: Cybercriminals are onto ...,"['Cybercrime', 'Startups']",CYBERCRIME,STARTUPS,NaN,"['Cybercrime', 'Startups']",0
4,0,0.0,World,Cybercrime's easiest prey: Small businesses,Small businesses are the 'most victimized' Cyb...,4/23/13 0:00,https://money.cnn.com/2013/04/22/smallbusiness...,NaN,CNN,Cybercrime's easiest prey: Small businesses,...,0.0,0.588,0.412,"[""Small businesses are the 'most victimized' C...",['Verizon'],VERIZON,NaN,NaN,['Verizon'],0


**PREPROCESSING**

Remove special characters using regex

In [ ]:
def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

In [ ]:
train["text"]=normalise_text(train["text"])

In [5]:
articles['Content_clean'] = articles['Text'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))

In [6]:
pd.set_option('display.max_colwidth', 200)
articles['Content_clean'].head(5)

0    A 20yearold man arrested in connection with a data breach that affected thousands of people in Germany has confessed to police, the countrys federal prosecutor, Georg Ungefuk, told reporters at a ...
1    Several Israeli government websites appeared to crash as antiIsraeli hackers launched cyberattacks Sunday, but Israeli hackers also claimed their own victory. The website that promoted the OpIsrae...
2    The Pentagon has accused China of trying to extract sensitive information from U.S. government computers, the latest in a series of rhetorical skirmishes between the two countries on the issue of ...
3    Startups take note: Cybercriminals are onto you. Cyberattackers can sniff out new businesses to target as quickly as two months after they come into existence, according to a new report from cyber...
4    Small businesses are the most victimized Cybercriminals have picked their easiest prey: Small businesses. A data breach investigations report from Verizon VZ, released Tuesday

Drop missing values in the three columns

In [7]:
df = pd.DataFrame(articles[['Level','Title','Content_clean']])
df = df.dropna(subset=['Level','Title','Content_clean'])

Count Vectorizer on Article Content

In [11]:
X = df['Content_clean']
y = df['Level']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7,random_state=10)
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# Modeling

### Fit a Naive Bayes Model and print evaluation metrics

In [17]:
# strictly unigram model
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print("accuracy score is: " + str(metrics.accuracy_score(y_test, y_pred_class)))
print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

accuracy score is: 0.8424710424710424
              precision    recall  f1-score   support

         0.0       0.90      0.97      0.93       984
         1.0       0.40      0.26      0.31       132
         2.0       0.70      0.60      0.64       179

    accuracy                           0.84      1295
   macro avg       0.66      0.61      0.63      1295
weighted avg       0.82      0.84      0.83      1295

[[950  21  13]
 [ 65  34  33]
 [ 41  31 107]]


In [8]:
# MultinomialNB test1
test_text="china has earthquake"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[2.]


In [10]:
# MultinomialNB test2
test_text="taylor swift has a new song called earthquake"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[0.]


In [11]:
# MultinomialNB test3
test_text="England has flood,1000 people died"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[2.]


In [12]:
# MultinomialNB test4
test_text="England has flood, no death"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[1.]


In [13]:
# MultinomialNB test5
test_text="there is a bomb in iran, no death"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[0.]


In [14]:
# MultinomialNB test6
test_text="there is a bomb in iran, causing 100 people death"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[1.]


In [23]:
# MultinomialNB test7
test_text="china and united states have trade war"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[0.]


In [18]:
# MultinomialNB test8
test_text="There is a war in iron, causing 1000 death"
test_text=[test_text]
test_text_dtm = vect.transform(test_text)
y_pred_class = nb.predict(test_text_dtm)
print(y_pred_class)

[1.]


TFIDF on Unigrams

In [11]:
count_vect.vocabulary_.get(u'algorithm')

In [19]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train) 
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

Re-run Naive Bayes Model

In [17]:
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
X_new_counts = count_vect.transform(X_test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred_class = nb.predict(X_new_tfidf)


CPU times: user 5.31 ms, sys: 1.32 ms, total: 6.63 ms
Wall time: 5.82 ms


0.7872340425531915

In [ ]:
print(metrics.accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))
predicted = nb.predict(X_new_tfidf)
np.mean(predicted ==  y_test)

Tune Hyperparameters 

In [18]:
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
 ])


In [19]:
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2), (1, 3)], # 
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),
}

In [20]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [21]:
gs_clf = gs_clf.fit(X_train, y_train)

In [23]:
#text_clf.fit(X_train,y_train)
#Pipeline(...)
predicted = gs_clf.predict(X_test)
np.mean(predicted ==  y_test)

0.8439716312056738

In [ ]:
# how to know what the best parameters are?


In [ ]:
# how to know what the best variables are?

### SVM 

In [24]:
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                            max_iter=5, tol=None)),
 ])

text_clf.fit(X_train,y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted ==  y_test)

0.8581560283687943

Tune Parameters

In [ ]:
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]

In [25]:
parameters = [
    {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
     'tfidf__use_idf': (True, False),
     'C': [1, 10, 100, 1000], 
     'kernel': ['linear']},
    {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
     'tfidf__use_idf': (True, False),
     'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}
]

In [26]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [27]:
gs_clf = gs_clf.fit(X_train, y_train)

In [28]:
predicted = gs_clf.predict(X_test)
np.mean(predicted ==  y_test)

0.8652482269503546

In [122]:
##X_new_counts = count_vect.transform(X_test)
#X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred_class =gs_clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

0.8652482269503546
              precision    recall  f1-score   support

         0.0       0.89      0.99      0.94       111
         1.0       0.38      0.23      0.29        13
         2.0       0.90      0.53      0.67        17

    accuracy                           0.87       141
   macro avg       0.72      0.58      0.63       141
weighted avg       0.85      0.87      0.85       141

[[110   1   0]
 [  9   3   1]
 [  4   4   9]]


In [100]:
# SVM test1
test_text="china has earthquake"
test_text=[test_text]
#test_text_dtm = vect.transform(test_text)
y_pred_class = text_clf.predict(test_text)
print(y_pred_class)

[2.]


In [101]:
# SVM test2
test_text="taylor swift has a new song called earthquake"
test_text=[test_text]
#test_text_dtm = vect.transform(test_text)
y_pred_class = text_clf.predict(test_text)
print(y_pred_class)

[0.]


Other improvement 

In [ ]:
#countvectorizer = CountVectorizer(stop_words='english')
#countvectorizer = CountVectorizer(max_df=0.5)
#countvectorizer = CountVectorizer(min_df=2)

In [30]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer(stop_words='english')),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                            max_iter=5, tol=None)),
 ])

parameters = {
     'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
     'vect__max_df': (0.1,0.2,0.5),
     'vect__min_df': (2,3),
     'tfidf__use_idf': (True, False)
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

gs_clf = gs_clf.fit(X_train, y_train)
predicted = gs_clf.predict(X_test)
np.mean(predicted ==  y_test)

text_clf.fit(X_train,y_train)
#Pipeline(...)
#predicted = text_clf.predict(X_test)
#np.mean(predicted ==  y_test)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabular...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [31]:
y_pred_class =gs_clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

0.8723404255319149
              precision    recall  f1-score   support

         0.0       0.91      0.97      0.94       111
         1.0       0.57      0.31      0.40        13
         2.0       0.73      0.65      0.69        17

    accuracy                           0.87       141
   macro avg       0.74      0.64      0.68       141
weighted avg       0.86      0.87      0.86       141

[[108   0   3]
 [  8   4   1]
 [  3   3  11]]


In [106]:
y_pred_class =gs_clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

0.8723404255319149
              precision    recall  f1-score   support

         0.0       0.91      0.97      0.94       111
         1.0       0.57      0.31      0.40        13
         2.0       0.73      0.65      0.69        17

    accuracy                           0.87       141
   macro avg       0.74      0.64      0.68       141
weighted avg       0.86      0.87      0.86       141

[[108   0   3]
 [  8   4   1]
 [  3   3  11]]
[[108   0   3]
 [  8   4   1]
 [  3   3  11]]


Other Testing as following--
Word N-grams, 
char n-grams，
char_wb grams，
Controlling features in CountVectorizer，
Sentiment Score as predictors,
Time as feature, 
Title combined with articles,
Other machine learning method,
(Feature Selections)

Dig in N-grams 

word n-grams

In [108]:
import sklearn
import pandas as pd
import wordcloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import metrics
import sys

In [109]:
subdf = df
X = subdf['Content_clean']
y = subdf.Level
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=10)
vect =  CountVectorizer(analyzer='word', ngram_range=(1,3))
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm
X_test_dtm = vect.transform(X_test)
X_test_dtm

<141x469046 sparse matrix of type '<class 'numpy.int64'>'
	with 210949 stored elements in Compressed Sparse Row format>

In [110]:
# Naive Bayes Model
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print(metrics.accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

CPU times: user 51.9 ms, sys: 26.3 ms, total: 78.3 ms
Wall time: 68.2 ms
0.851063829787234
[[110   0   1]
 [  9   3   1]
 [  3   7   7]]


In [112]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
     ('vect', CountVectorizer(analyzer='word', ngram_range=(1,3))),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
 ])

#text_clf.fit(X_train,y_train)
#Pipeline(...)
#predicted = text_clf.predict(X_test)
#np.mean(predicted ==  y_test)
from sklearn.model_selection import GridSearchCV
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2),(1,3)],
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),
     }
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_train,y_train)
gs_clf.best_score_
#gs_clf

0.8502673796791443

Controlling features in CountVectorizer

In [113]:
subdf = df
X = subdf['Content_clean']
y = subdf.Level
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=10)
vect =  CountVectorizer(lowercase=False, stop_words='english',
                                  max_df=0.8, min_df=0.2, max_features=1000, ngram_range=(1,3))
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm
X_test_dtm = vect.transform(X_test)
X_test_dtm

# Naive Bayes Model
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print(metrics.accuracy_score(y_test, y_pred_class))
#print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

CPU times: user 999 µs, sys: 3 µs, total: 1 ms
Wall time: 1.01 ms
0.5390070921985816
[[58 35 18]
 [ 2  4  7]
 [ 1  2 14]]


char n-grams

In [114]:
subdf = df
X = subdf['Content_clean']
y = subdf.Level
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=10)
vect = countvectorizer = CountVectorizer(analyzer='char', ngram_range=(1,3))
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm
X_test_dtm = vect.transform(X_test)
X_test_dtm

# Naive Bayes Model
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print(metrics.accuracy_score(y_test, y_pred_class))
#print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

CPU times: user 7.77 ms, sys: 2.4 ms, total: 10.2 ms
Wall time: 9.86 ms
0.574468085106383
[[63 43  5]
 [ 2  8  3]
 [ 0  7 10]]


char_wb grams

In [115]:
subdf = df
X = subdf['Content_clean']
y = subdf.Level
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=10)
vect = countvectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(1,3))
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm
X_test_dtm = vect.transform(X_test)
X_test_dtm

# Naive Bayes Model
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print(metrics.accuracy_score(y_test, y_pred_class))
#print(classification_report(y_test, y_pred_class))
print(metrics.confusion_matrix(y_test, y_pred_class))

CPU times: user 6.68 ms, sys: 2.03 ms, total: 8.71 ms
Wall time: 8.71 ms
0.6028368794326241
[[67 40  4]
 [ 2  8  3]
 [ 0  7 10]]
